In [1]:
import pandas as pd
import numpy as np
import psycopg2
import warnings
warnings.filterwarnings('ignore')

print("Bibliotecas importadas com sucesso!")


Bibliotecas importadas com sucesso!


## 2. Conexão com PostgreSQL


In [2]:
DB_CONFIG = {
    'host': 'postgres',
    'port': 5432,
    'database': 'inmet_db',
    'user': 'inmet_user',
    'password': 'inmet_password'
}

print("Configuração de conexão definida!")


Configuração de conexão definida!


## 3. Carregamento de Dados para Visualização


In [3]:
# Carrega dados com informações de estações
query = """
SELECT 
    dm.timestamp_utc,
    dm.codigo_wmo,
    e.nome as nome_estacao,
    e.uf,
    e.latitude,
    e.longitude,
    dm.precipitacao_mm,
    dm.intensidade_chuva,
    dm.temperatura_ar_c,
    dm.umidade_rel_horaria_pct,
    dm.pressao_estacao_mb,
    dm.vento_velocidade_ms,
    dm.radiacao_global_kjm2
FROM dados_meteorologicos dm
JOIN estacoes e ON dm.codigo_wmo = e.codigo_wmo
WHERE dm.intensidade_chuva IS NOT NULL
ORDER BY dm.timestamp_utc
"""

# Lê dados usando conexão direta com psycopg2 (evita problemas de encoding)
try:
    conn = psycopg2.connect(
        host=DB_CONFIG['host'],
        port=DB_CONFIG['port'],
        database=DB_CONFIG['database'],
        user=DB_CONFIG['user'],
        password=DB_CONFIG['password']
    )
    conn.set_client_encoding('UTF8')
    df = pd.read_sql(query, conn)
    conn.close()
    
    df['timestamp_utc'] = pd.to_datetime(df['timestamp_utc'])
    print("Dados carregados com sucesso!")
except Exception as e:
    print(f"Erro ao carregar dados: {e}")
    df = pd.DataFrame()

if len(df) > 0:
    print(f"Dados carregados: {len(df):,} registros")
    print(f"Período: {df['timestamp_utc'].min()} até {df['timestamp_utc'].max()}")
    print(f"Estações: {df['codigo_wmo'].nunique()}")
    display(df.head())
else:
    print("Nenhum dado disponível!")


Dados carregados com sucesso!
Nenhum dado disponível!
